In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
cwd=os.getcwd()

from oligo_designer_toolsuite.oligo_specificity_filter._filter_base import ProbeFilterBase
from oligo_designer_toolsuite.oligo_specificity_filter._filter_blastn import ProbeFilterBlastn
from oligo_designer_toolsuite.oligo_specificity_filter._filter_bowtie import ProbeFilterBowtie
from oligo_designer_toolsuite.oligo_specificity_filter._filter_bowtie2 import ProbeFilterBowtie2
from oligo_designer_toolsuite.IO._data_parser import read_oligos_DB_tsv, write_oligos_DB_tsv

In [5]:
cwd

'/home/chels/CRISPR/repos/oligo-designer-toolsuite/tests'

In [148]:
n_jobs=8
ligation_region=10
dir_output=cwd + "/output"
dir_annotations=cwd+"/data"
file_probe_info=dir_annotations + "/oligo_DB_unknown_unknown_Custom_release_unknown_gene_transcript.tsv"
min_probes_per_gene=2
file_transcriptome_fasta=dir_annotations +"/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50


In [149]:
probe_info_dict=read_oligos_DB_tsv(file_probe_info)

Test Blast filter

In [43]:
blast_filter=ProbeFilterBlastn(n_jobs,
        file_transcriptome_fasta, 
        dir_output, dir_annotations, word_size,
        percent_identity, coverage, probe_length_min, probe_length_max, ligation_region)

In [44]:
blast_filter.apply(probe_info_dict)

INFO:filter_probes:Creating batches
INFO:filter_probes:Exact matches filtered in 6.1890411999993376 seconds
INFO:filter_probes:Creating blast database
INFO:filter_probes:Blast database created in 23.24813390000054 seconds
INFO:filter_probes:Blast alignmnet search finished in 157.10336969999844 seconds
INFO:filter_probes:Blast results processed in 11.953025200000411 seconds


Test Bowtie filter

In [220]:
n_jobs=10
min_mismatches=4
mismatch_region=5
ligation_region=7

bowtie_filter=ProbeFilterBowtie(n_jobs,
        dir_output,dir_annotations,
        file_transcriptome_fasta, min_mismatches, mismatch_region, ligation_region)



In [37]:
bowtie_filter.dir_bowtie

'/home/chels/CRISPR/repos/oligo-designer-toolsuite/tests/output/bowtie'

In [208]:
bowtie_filter.apply(probe_info_dict)
#find a probe with mismatch in ligation region and run on that. 




INFO:filter_probes:Creating batches
INFO:filter_probes:Exact matches filtered in 4.182710199995199 seconds


# reads processed: 2960
# reads with at least one reported alignment: 2960 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 4038 alignments to 1 output stream(s)
# reads processed: 5736
# reads with at least one reported alignment: 5736 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 10770 alignments to 1 output stream(s)
# reads processed: 5284
# reads with at least one reported alignment: 5284 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 5920 alignments to 1 output stream(s)
# reads processed: 59
# reads with at least one reported alignment: 59 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 59 alignments to 1 output stream(s)
# reads processed: 2913
# reads with at least one reported alignment: 2913 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 3638 alignments to 1 output stream(s)
# reads processed: 3235
# reads with at least one reported alignment: 3235 (100.00%)
# reads that failed to align: 0 (0.00%)
Reported 30205 align

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/chels/anaconda3/envs/oligo/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4168/1462788976.py", line 1, in <cell line: 1>
    bowtie_filter.apply(probe_info_dict)
  File "/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/oligo_specificity_filter/_filter_bowtie.py", line 310, in apply
    self.run_bowtie()
  File "/home/chels/CRISPR/repos/oligo-designer-toolsuite/oligo_designer_toolsuite/oligo_specificity_filter/_filter_bowtie.py", line 136, in run_bowtie
    Parallel()(
  File "/home/chels/anaconda3/envs/oligo/lib/python3.10/site-packages/joblib/parallel.py", line 1056, in __call__
    self.retrieve()
  File "/home/chels/anaconda3/envs/oligo/lib/python3.10/site-packages/joblib/parallel.py", line 935, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/home/chels/anaconda3/env

In [233]:
#Get mismatches in ligation region for bowtie

import pandas as pd
bowtie_results_matches=pd.read_csv("output/bowtie/bowtie_batch0_0.txt", sep="\t", header=None)

#Get probes where mismatch positions are given
bowtie_results_matches_mismatch_positions=bowtie_results_matches[bowtie_results_matches[7].notna()]
#Extract positions
positions=bowtie_results_matches_mismatch_positions[7].str.split(",").apply(lambda x: [elem.split(":")[0] for elem in x])
#add positions to bowtie results 
bowtie_results_matches_positions = bowtie_results_matches.merge(
                    positions,
                    left_index=True,
                    right_index=True,
                    how="left"
                )
                
probes_info=pd.read_csv("data/probes_info_batch0.txt", sep="\t", header=0)
bowtie_results_add_ligation = bowtie_results_matches_positions.merge(
    probes_info[["probe_id", "ligation_site"]],
    left_on=0,
    right_on="probe_id",
    how="inner",
)

#Calculate ligation region
ligation_region=10
bowtie_results_add_ligation['ligation_region_start']=bowtie_results_add_ligation.ligation_site-(ligation_region-1)
bowtie_results_add_ligation['ligation_region_end']=bowtie_results_add_ligation.ligation_site+ligation_region

bowtie_results_add_ligation.rename(columns={"7_y":"positions"}, inplace=True)
bowtie_results_add_ligation['mismatch_in_ligation']=[False for i in range(len(bowtie_results_add_ligation))]
bowtie_results_add_ligation_notna=bowtie_results_add_ligation[bowtie_results_add_ligation["positions"].notna()]

def mismatch_in_ligation(row):
    mismatches_in_region=[]
    for mismatch in row["positions"]:
        if int(mismatch) >= int(row["ligation_region_start"]) and int(mismatch) <= int(row["ligation_region_end"]):
            mismatches_in_region.append(True)
    if len(mismatches_in_region)>0:
        row['mismatch_in_ligation']=mismatches_in_region[0]
        row["num_mismatches_in_ligation"] = len(mismatches_in_region)
    return row

bowtie_results_add_ligation_notna=bowtie_results_add_ligation_notna.apply(mismatch_in_ligation, axis=1)

Test Bowtie 2 filter

In [5]:

bowtie2_filter=ProbeFilterBowtie2(n_jobs,
        dir_output,dir_annotations,
        file_transcriptome_fasta)

In [6]:
bowtie2_filter.apply(probe_info_dict)

INFO:filter_probes:Creating batches
INFO:filter_probes:Exact matches filtered in 8.197780699999385 seconds
Settings:
  Output files: "transcriptome_bowtie2.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/chels/CRISPR/repos/oligo-designer-toolsuite/tests/data/reference_DB_unknown_unknown_Custom_release_unknown_genome_False_gene_transcript_True
Reading reference sizes


Building a SMALL index


  Time reading reference sizes: 00:00:03
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:02
bmax according to bmaxDivN setting: 49779804
Using parameters --bmax 37334853 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 37334853 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:08
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:03
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:03
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 12 samples
  (

Killed
(ERR): bowtie2-align exited with value 137
59 reads; of these:
  59 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    59 (100.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
100.00% overall alignment rate
2960 reads; of these:
  2960 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    2134 (72.09%) aligned exactly 1 time
    826 (27.91%) aligned >1 times
100.00% overall alignment rate
2913 reads; of these:
  2913 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    2180 (74.84%) aligned exactly 1 time
    733 (25.16%) aligned >1 times
100.00% overall alignment rate
5284 reads; of these:
  5284 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    4618 (87.40%) aligned exactly 1 time
    666 (12.60%) aligned >1 times
100.00% overall alignment rate
3235 reads; of these:
  3235 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    0 (0.00%) aligned exactly 1 time
    3235 (100.00%) aligned 

INFO:filter_probes:Bowtie2 alignmnet search finished in 39.63208680000025 seconds


8190 reads; of these:
  8190 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    6567 (80.18%) aligned exactly 1 time
    1623 (19.82%) aligned >1 times
100.00% overall alignment rate


INFO:filter_probes:Bowtie2 results processed in 3.2599476999994295 seconds
